### Import

In [1]:
from datasets import  Value, Sequence,Features,load_dataset,DatasetDict, Dataset,concatenate_datasets
import numpy as np
import evaluate
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments, Trainer, DataCollatorForTokenClassification,BertTokenizerFast,AutoModelForTokenClassification

/Users/matthieudecotignie/opt/miniconda3/envs/RoboDoc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-19 17:29:11.376143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load DataSet and define the labels

In [2]:
import csv
import os
word_dict = {}
word_dict['tokens']=[]
word_dict['tags']=[]
csv_file_path = os.curdir+'/dataset.csv' 
txt_file_path = os.curdir+'/1-1000.txt'

def extract_words_txt(txt_file):
  with open(txt_file) as f:
    lines = f.readlines()
    for word in lines:
                  word = word.strip().replace('_',' ')  
                  if word and word not in word_dict['tokens'] and word!=',':
                       word_dict['tokens'].append(word)
                       word_dict['tags'].append('0')


def extract_words(csv_file):
   
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for idl, line in enumerate(reader):
            if(idl!=0):
                for id,word in enumerate(line):
                  word = word.strip().replace('_',' ')  
                  if word and word not in word_dict['tokens'] and id!=0 and word!=',':
                       word_dict['tokens'].append( word)
                       word_dict['tags'].append('B-Sign_symptom')
        


extract_words(csv_file_path)
extract_words_txt(txt_file_path)

print(len(word_dict['tokens']))

1114


In [3]:

dataset2 = Dataset.from_dict(word_dict)
dataset1=load_dataset("ktgiahieu/maccrobat2018_2020")['train']

dataset2_sequences = Dataset.from_dict({
    "tokens": [[token] for token in dataset2["tokens"]],
    "tags": [[tag] for tag in dataset2["tags"]]
})

ds= concatenate_datasets([dataset1,dataset2_sequences])

print(ds)
label_list = ["O",
"B-Age",
"I-Age",
"B-Sex","I-Sex",
"B-Disease_disorder",
"I-Disease_disorder",
"B-Sign_symptom",
"I-Sign_symptom",
'I-Biological_structure','B-Biological_structure','I-Lab_value','B-Lab_value']

label_encoding_dict = {"O":0,
"B-Age":1,
"I-Age":2,
"B-Sex":3,
"I-Sex":4,
"B-Disease_disorder":5,
"I-Disease_disorder":6,
"B-Sign_symptom":7,
"I-Sign_symptom":8,
'I-Biological_structure':9,'B-Biological_structure':10,'I-Lab_value':11,'B-Lab_value':12}

Found cached dataset json (/Users/matthieudecotignie/.cache/huggingface/datasets/ktgiahieu___json/ktgiahieu--maccrobat2018_2020-768e4d068e4e6de6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 16.80it/s]

Dataset({
    features: ['tokens', 'tags'],
    num_rows: 1514
})


### Tokenize the data set using the sciBERT tokenizer and then split between training and test set

In [4]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

In [5]:


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], is_split_into_words=True,truncation =True,padding=True,max_length=512)
   
    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx !=previous_word_idx:
                label_ids.append(label_encoding_dict.get(label[word_idx],0))
            else:
                label_ids.append(label_encoding_dict.get(label[word_idx],0))
            previous_word_idx = word_idx
        labels.append(label_ids) 
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets_train = ds.shuffle(seed = 42).select(range(1200)).map(tokenize_and_align_labels, batched=True, batch_size=20)

tokenized_datasets_test = ds.shuffle(seed = 42).select(range(1200,1514)).map(tokenize_and_align_labels, batched=True, batch_size=20)

Loading cached shuffled indices for dataset at /Users/matthieudecotignie/.cache/huggingface/datasets/ktgiahieu___json/ktgiahieu--maccrobat2018_2020-768e4d068e4e6de6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-da61ed3c9e0c4be5.arrow


### Compute Metrics

In [6]:
metric = evaluate.load("seqeval")

def compute_metrics(eval_preds): 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Load the model, define training arguments and train the model 

In [7]:
model = AutoModelForTokenClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=len(label_list))

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [17]:
data_collator = DataCollatorForTokenClassification(tokenizer)

batch_size = 16

args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=batch_size,
per_device_eval_batch_size=batch_size,
num_train_epochs=15,
logging_steps=1
) 

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.284800,0.319027,0.497093,0.452830,0.473930,0.889975
2,0.138400,0.247670,0.544610,0.543529,0.544069,0.905044
3,0.246100,0.222509,0.584932,0.706720,0.640084,0.917343
4,0.001200,0.192842,0.698418,0.643165,0.669654,0.932689
5,0.071500,0.170392,0.674479,0.771599,0.719778,0.941220
6,0.104800,0.165404,0.664684,0.813638,0.731656,0.942993
7,0.024800,0.159174,0.677437,0.832837,0.747142,0.945348
8,0.186200,0.139214,0.716060,0.820589,0.764769,0.955791
9,0.210600,0.122127,0.781513,0.831182,0.805582,0.964212
10,0.044300,0.120319,0.780036,0.863952,0.819852,0.966289


TrainOutput(global_step=2250, training_loss=0.12933121185942179, metrics={'train_runtime': 1834.5292, 'train_samples_per_second': 9.812, 'train_steps_per_second': 1.226, 'total_flos': 4703809370112000.0, 'train_loss': 0.12933121185942179, 'epoch': 15.0})

In [8]:
saving_path=os.curdir+"/ner_model"


In [ ]:
model.save_pretrained(saving_path)

In [9]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained(saving_path)

In [10]:
from transformers import pipeline
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer,aggregation_strategy="simple")


example = " I have a massive migraine since last week, my nose is runny and I might have some fever "
ner_results = nlp(example)

print(ner_results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LABEL_0', 'score': 0.96986866, 'word': 'i have a massive', 'start': 1, 'end': 17}, {'entity_group': 'LABEL_7', 'score': 0.97137713, 'word': 'migraine', 'start': 18, 'end': 26}, {'entity_group': 'LABEL_0', 'score': 0.9160592, 'word': 'since last week, my nose is', 'start': 27, 'end': 54}, {'entity_group': 'LABEL_7', 'score': 0.98303044, 'word': 'runny', 'start': 55, 'end': 60}, {'entity_group': 'LABEL_0', 'score': 0.89259624, 'word': 'and i might have some', 'start': 61, 'end': 82}, {'entity_group': 'LABEL_7', 'score': 0.9946254, 'word': 'fever', 'start': 83, 'end': 88}]
